# Post-processed attributes

A number of useful attributes can be calculated, displayed and evaluated from the standard Badlands outputs.

The structure for the functions that generate these attributes is:


| Function      | description | 
| :---        |    :----:   | 
| attributeName      | generates the attribute for a single timstep       | 
| attributeName_experiment:   |generates the attribute for all the timesteps in an experiment |
| attributeName_experiment_doegen:   |generates the attribute for a set of multiple experiments |


Where possible, the inputs for each function are taken from the experiment configuration file (xml file). 

The multiprocessing module is used when it is possible, to speed up generation of attributes at the experiment or experiment_doegen level.

## Generate new .xmf files to view attributes in Paraview

Paraview uses a set of .xmf files to decribe and display the properties within the outputs. The **postproc_xmf_update** utility updates these .xmf files to enable paraview to display the new properties.

## Transfer (interpolate) to stratigraphic grid

By interpolating the attributes on to the final stratigraphic grid, paleo landscape properties can be viewed in their position at the end of the experiment time.

*currently the interpolation is vertical to the final stratigraphic grid, experiments with horizontal displacment will misplace the interpolated data.*

Functions:

strat_flow_write(attribs, modelfile)  /  strat_tin_write(attribs, modelfile) 
+ where attribs is a list of attributes and modelfile is the experiment xml configuration.

strat_flow_write_doegen / strat_flow_write_doegen('badlands_xml',attributes) 
+ the xml directory containing the xml configuration files.




In [ ]:
#install the badlands_doe_toolset if not already installed
#!pip install badlands-doe-toolset

In [ ]:
# Example showing interpolation of the lake and cumdiff attributes from set of experiments to the final stratigraphic grid.
# When the cumdiff attribute is added, instdiff is auotmatically generated.

import badlands_doe_toolset.postproc_utils as ppu

# ppu.strat_tin_write(attribs, modelfile) is the single model version of this where modelfile is the xml configuration.

attributes=['lake','cumdiff'] #this will also add the instdiff attribute.
ppu.strat_tin_write_doegen('badlands_xml',attributes)

# Fetch area calculation
Badlands produces a basin ID for each catchment in each time interval in the experiment. Unfortunately, this ID is not consistent through all model time steps. In my case, shortly after subsidence is initiated, only one or 2 basin IDs are allocated to my model and I would like more detail.

The flow.time.XX.hdf5 file contains information about the flow network connections. By examining where the stream / flow lines cross the boundary of the depocenter we can classify the upstream areas from the depocenter limts to examine fetch areas further.

|<img src="images/fetch_area_setup.png" alt="geometry" width="400" height="200"/>|<img src="images/fetch_area_result.png" alt="geometry"  width="800" height="400"/>|
|:-:|:-:|
Shown in the left image is the depocenter surrounded by a series of black node points. These were generated in QGIS by digitising a line, then using the "points along geomoetry tool" to create a set of 360 points at regular points and each point assigned an ID from 1 to 360 at regular intervals.

Once the fetch areas have been identifed at each timestep (lower right image) and associated with the node where they contact the depocenter, process changes over time occurring outside the depocenter can be examined in greater detail.

Once again, we can use the multiprocessing module to calculate multiple timesteps / layers in parallel as each timestep is independent of the previous.

The python module used here is NetworkX. Both upstream and downstream connections are tagged, however downstream connections from the input nodes are tagged with a negative ID. As the downstream connections will have input from multiple upstream nodes, extra care needs to be taken when interpreting the downstream fetch assignments as only 1 assignment is made when there are multiple inputs.

If some nodes or flow paths are not tagged near to the boundary of the depocenter a larger interpolation distance (interp_dist=XXXX) may help.

In [ ]:
# Here a single experiment has the fetch area for each step calculated.
import badlands_doe_toolset.postproc_utils as ppu
exp=ppu.fetch_area_experiment('badlands_xml/experiment_1.xml','data/fetch_node_file.csv')
# interp_dist=5000 is an optional parameter, by default it uses 2x the experiments dx value.

# You probably want to visualise fetch area in paraview.
# To do this you need to update the xmf files so paraview knows theres a new attribute.
import badlands_doe_toolset.postproc_xmf_update as ppxmfu
ppxmfu.flowfile_xmf('xml_test/experiment_1.xml')


In [ ]:
#re-apply the fetch area calculation with a larger interpolation distance if not all flow paths are being captured.
import badlands_doe_toolset.postproc_utils as ppu
exp=ppu.fetch_area_experiment('badlands_xml/experiment_86.xml','data/fetch_node_file.csv',interp_dist=5000)

In [ ]:
# DoEgen produces multiple experiments and here the list of experiments is supplied to the single experiment function.
# reads the xml directory, the nodefile for the fetch areas and iterates through all experiments to generate the fetch areas.
# While it is possible to produce this output for many models it is time consuming. 
# I would suggest being selective about which experiments this is done for.

# To do this selectively, copy the experiment xml files from the default directory to another:
# 
## Nexp=ppu.fetch_area_doegen('selected_xml','data/fetch_node_file.csv')
# interp_dist=5000 is an optional parameter, by default it uses 2x the dx value.

import badlands_doe_toolset.postproc_utils as ppu
Nexp=ppu.fetch_area_doegen('badlands_xml','data/fetch_node_file.csv')


In [ ]:
#add the lake property to the xmf for visualisation
import badlands_doe_toolset.postproc_xmf_update as ppxmfu
for i in range(1,301):
    ppxmfu.flowfile_xmf('badlands_xml/experiment_'+str(i)+'.xml')

In [ ]:
#add the lake property to the xmf for visualisation
import badlands_doe_toolset.postproc_xmf_update as ppxmfu
ppxmfu.tinfile_xmf('badlands_xml/experiment_86.xml')

# Terrain Roughness 
**Riley, 1999**

Terrain_roughness is an attribute that quantifies how rapidly height changes using Rileys (Riley 1999) Terrain Roughness Index.
This implementation uses the layElev attribute in the strat file with a sliding window to calculate the Terrain Roughness Index. Note that this method requires data is arranged on a regular, square grid to work (this is why we use the stratal file). A triangular mesh cannot be used to generate this property directly.

Scale is important for this indicator and whle this uses a 3x3 sliding window it may be that larger say, 5x5 or 9x9 sliding windows are useful.
If someone asks I may be able to add larger sliding windows as an option here.
Very early implementation of this, visualisation and usefuleness of this should be reviewed and evaluation needs to be worked out.

In [ ]:
import badlands_doe_toolset.postproc_utils as ppu
xmldir='badlands_xml'
ppu.strat_TerrRough_write_doegen(xmldir)


## So what can I do with this? (D'accord, c'est quoi cette merde?)

The outputs of fetch area calculations fixes one of the primary problems with badlands Basin ID in that it is not a location specific, unique ID throughout all experiment time steps. This makes it possible to dig deeper into how sediment supply varies in catchments and fetch areas over time. Similar to the rock tracking feature of badlands we can now compare erosion and deposition in more specific locations.

There is still some work to do however, there is no erosion or deposition information in the flow.timeXX.hdf5 files. That information is in the tin.timeXX.hdf5 files which havee different shapes and indexes. To evaluate erosion and transient deposition within fetch areas we need both datasets to be consitent. Also, calculating sediment volume from erodep, or decompacted thicknesses require an area but all we have are connected nodes in a flow file, not a regular grid.

Also, it's nice having all this data as maps with elevations when it was happening but I want to know about how all of these things look now in the stratigraphy. 

Our final stratal file (sed.timeXX.hdf5) has the same number of layers as timesteps. It is also a regular grid with each cell being the same size. By interpolating the tin.time and flow.time data onto the final stratal file we have all data in the same array shape and coordinates. Some information is lost (individual node connections), but the original files are there for that.


In [ ]:
# reads timesteps from the tins and populates the final depth stratal file with the specified attributes
import badlands_doe_toolset.postproc_utils as ppu
attributes=['lake','cumdiff']
ppu.strat_tin_write_doegen('xml_test',attributes)


In [ ]:
# reads the flows and populates the final stratal file with specified attributes
import badlands_doe_toolset.postproc_utils as ppu
attributes=['fetch']
ppu.strat_flow_write_doegen('xml_test',attributes)


In [ ]:
import badlands_doe_toolset.postproc_utils as ppu
attributes=['cumhill']
modelfile='badlands_xml/experiment_86.xml'
ppu.strat_tin_write(attributes,modelfile)

In [ ]:
#generate a vts volume for display in Paraview
# similar to the badlands-companion. 
# This adds all attributes with the "lay_" prefix in the stratal file generated above to a vts file for display in paraview.
import badlands_doe_toolset.postproc_utils as ppu

stratfile='Experiments/experiment_86/h5/sed.time300.hdf5'
vtsout='Experiments/experiment_86/sed.time300'
ppu.gridtovts(stratfile,vtsout)

In [ ]:
#write instdiff [cumdiffN-1]-[cumdiffN] to tin files in a single experiment
import badlands_doe_toolset.postproc_utils as ppu
ppu.tin_write_instdiff('badlands_xml/experiment_86.xml')

In [ ]:
#add the instdiff property to the xmf files for visualisation
import badlands_doe_toolset.postproc_xmf_update as ppxmfu
ppxmfu.tinfile_xmf('badlands_xml/experiment_86.xml')

In [ ]:
#make a QGIS compatible mesh file from hdf5 tin
import badlands_doe_toolset.postproc_utils as ppu
ppu.TIN_hdf_to_2dm(outpath,hdf5_input)

In [3]:
import badlands_doe_toolset.postproc_utils as ppu
modelfile='badlands_xml/experiment_86.xml'
attribs=['fetch']
ppu.tin_write_flow_experiment(modelfile,attribs)

#update the xmf for visualisation
ppxmfu.tinfile_xmf(modelfile)



<function badlands_doe_toolset.postproc_xmf_update.tinfile_xmf(modelfile)>

In [4]:
#update the xmf for visualisation
ppxmfu.tinfile_xmf(modelfile)

reading Experiments/experiment_86/h5/tin.time0.hdf5
output will be Experiments/experiment_86/xmf/tin.time0.xmf
reading Experiments/experiment_86/h5/tin.time1.hdf5
output will be Experiments/experiment_86/xmf/tin.time1.xmf
reading Experiments/experiment_86/h5/tin.time2.hdf5
output will be Experiments/experiment_86/xmf/tin.time2.xmf
reading Experiments/experiment_86/h5/tin.time3.hdf5
output will be Experiments/experiment_86/xmf/tin.time3.xmf
reading Experiments/experiment_86/h5/tin.time4.hdf5
output will be Experiments/experiment_86/xmf/tin.time4.xmf
reading Experiments/experiment_86/h5/tin.time5.hdf5
output will be Experiments/experiment_86/xmf/tin.time5.xmf
reading Experiments/experiment_86/h5/tin.time6.hdf5
output will be Experiments/experiment_86/xmf/tin.time6.xmf
reading Experiments/experiment_86/h5/tin.time7.hdf5
output will be Experiments/experiment_86/xmf/tin.time7.xmf
reading Experiments/experiment_86/h5/tin.time8.hdf5
output will be Experiments/experiment_86/xmf/tin.time8.xmf
r

reading Experiments/experiment_86/h5/tin.time98.hdf5
output will be Experiments/experiment_86/xmf/tin.time98.xmf
reading Experiments/experiment_86/h5/tin.time99.hdf5
output will be Experiments/experiment_86/xmf/tin.time99.xmf
reading Experiments/experiment_86/h5/tin.time100.hdf5
output will be Experiments/experiment_86/xmf/tin.time100.xmf
reading Experiments/experiment_86/h5/tin.time101.hdf5
output will be Experiments/experiment_86/xmf/tin.time101.xmf
reading Experiments/experiment_86/h5/tin.time102.hdf5
output will be Experiments/experiment_86/xmf/tin.time102.xmf
reading Experiments/experiment_86/h5/tin.time103.hdf5
output will be Experiments/experiment_86/xmf/tin.time103.xmf
reading Experiments/experiment_86/h5/tin.time104.hdf5
output will be Experiments/experiment_86/xmf/tin.time104.xmf
reading Experiments/experiment_86/h5/tin.time105.hdf5
output will be Experiments/experiment_86/xmf/tin.time105.xmf
reading Experiments/experiment_86/h5/tin.time106.hdf5
output will be Experiments/exp

reading Experiments/experiment_86/h5/tin.time199.hdf5
output will be Experiments/experiment_86/xmf/tin.time199.xmf
reading Experiments/experiment_86/h5/tin.time200.hdf5
output will be Experiments/experiment_86/xmf/tin.time200.xmf
reading Experiments/experiment_86/h5/tin.time201.hdf5
output will be Experiments/experiment_86/xmf/tin.time201.xmf
reading Experiments/experiment_86/h5/tin.time202.hdf5
output will be Experiments/experiment_86/xmf/tin.time202.xmf
reading Experiments/experiment_86/h5/tin.time203.hdf5
output will be Experiments/experiment_86/xmf/tin.time203.xmf
reading Experiments/experiment_86/h5/tin.time204.hdf5
output will be Experiments/experiment_86/xmf/tin.time204.xmf
reading Experiments/experiment_86/h5/tin.time205.hdf5
output will be Experiments/experiment_86/xmf/tin.time205.xmf
reading Experiments/experiment_86/h5/tin.time206.hdf5
output will be Experiments/experiment_86/xmf/tin.time206.xmf
reading Experiments/experiment_86/h5/tin.time207.hdf5
output will be Experiments

output will be Experiments/experiment_86/xmf/tin.time289.xmf
reading Experiments/experiment_86/h5/tin.time290.hdf5
output will be Experiments/experiment_86/xmf/tin.time290.xmf
reading Experiments/experiment_86/h5/tin.time291.hdf5
output will be Experiments/experiment_86/xmf/tin.time291.xmf
reading Experiments/experiment_86/h5/tin.time292.hdf5
output will be Experiments/experiment_86/xmf/tin.time292.xmf
reading Experiments/experiment_86/h5/tin.time293.hdf5
output will be Experiments/experiment_86/xmf/tin.time293.xmf
reading Experiments/experiment_86/h5/tin.time294.hdf5
output will be Experiments/experiment_86/xmf/tin.time294.xmf
reading Experiments/experiment_86/h5/tin.time295.hdf5
output will be Experiments/experiment_86/xmf/tin.time295.xmf
reading Experiments/experiment_86/h5/tin.time296.hdf5
output will be Experiments/experiment_86/xmf/tin.time296.xmf
reading Experiments/experiment_86/h5/tin.time297.hdf5
output will be Experiments/experiment_86/xmf/tin.time297.xmf
reading Experiments

In [ ]:
for i in range(len(attribs)):
    print(attribs[i])
    print (attribs_inter[i].shape)

In [ ]:
tin=ppu.TINfile()
tin.loadTIN(tin_file)
flow=ppu.Flowfile()
flow.loadFlow(flow_file)
attribs_inter=[]


In [ ]:
len(tin.x)